In [ ]:
import sys

sys.path.insert(0, '/media/saurabh/New Volume/tf_audio_sentiment/pyAudioAnalysis')

from  pyAudioAnalysis  import audioBasicIO
from  pyAudioAnalysis import audioFeatureExtraction

In [2]:
import tensorflow as tf
from tensorflow.contrib import rnn

import numpy as np
import os

unhappy_directory_path='/media/saurabh/New Volume/tf_audio_sentiment/data_test/data/0/'
unhappy_directory = os.fsencode(unhappy_directory_path)

happy_directory_path='/media/saurabh/New Volume/tf_audio_sentiment/data_test/data/1/'
happy_directory = os.fsencode(happy_directory_path)

model_path = "/media/saurabh/New Volume/tf_audio_sentiment/models/rnn_model.ckpt"

In [3]:
def audio_to_features(fileurl):
    
    [Fs, x] = audioBasicIO.readAudioFile(fileurl);
    F = audioFeatureExtraction.stFeatureExtraction(x, Fs, 0.050*Fs, 0.025*Fs);
    
    a=F[:21,:]
    b=F[33]

    b=np.expand_dims(b, axis=0)

    #print(b.shape)


    d = np.concatenate( (a, b), axis=0)
    #print(d.shape)
    d = np.swapaxes(d, 0, 1)
    return d

In [4]:
# Parameters
learning_rate = 0.001
training_epochs = 25
batch_size = 2
display_step = 1

In [5]:
timesteps=2000 # max timesteps
num_input = 22 # size of vector, at each timestep
num_classes = 2 # happy or sad

num_hidden = 110 # hidden layer num of features

x = tf.placeholder(tf.float32, [None,timesteps, num_input]) 
y = tf.placeholder(tf.int32, [None]) #  Index of output , 0 = sad , 1 = happy

In [6]:
# define a function, that pads audio so that audio frames = max frames

def pad(input):
   # print(input.shape[0])
     if input.shape[0] < timesteps:
        
        diff = timesteps - input.shape[0]
        
        # pad and return input
        return np.pad(input,((0,diff),(0,0)), mode="constant")
    
     elif input.shape[0] > timesteps:
        
        return input[:timesteps,:]
        

In [7]:
# Define weights
weights = {
    # Hidden layer weights => 2*n_hidden because of forward + backward cells
    'out': tf.Variable(tf.random_normal([2*num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}


In [8]:
def BiRNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, num_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, num_input)
    x = tf.unstack(x, timesteps, 1)

    # Define lstm cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    try:
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                              dtype=tf.float32)
    except Exception: # Old TensorFlow version only returns outputs not states
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                        dtype=tf.float32)
        
    # Linear activation, using rnn inner loop last output
    #return tf.matmul(outputs[-1], weights['out']) + biases['out']
    return outputs[-1]

In [9]:
Logits = BiRNN(x, weights, biases)
# prediction = tf.nn.softmax(logits) # use only for inference, softmax included in loss function


In [10]:
# Minimize error using cross entropy
#cost = tf.reduce_mean(-tf.reduce_sum(y_one_hot*tf.log(pred), reduction_indices=1))

cost = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=Logits)

# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [11]:
# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()

In [14]:
# Start training
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        
        
         # Loop over all unhappy files
        iterator=iter(os.listdir(unhappy_directory))
        for file in iterator:
            filename = os.fsdecode(file)
            
            
            if filename.endswith(".wav"): 
                accoustic_features=np.zeros(shape=(batch_size,2000,22))
                Y=np.zeros(shape=(batch_size))
                
                accoustic_features[0]=pad(audio_to_features(unhappy_directory_path + filename))
                print(filename)
                
                for i in range(batch_size - 1):
                    current_file=os.fsdecode(next(iterator))
                    accoustic_features[i]=pad(audio_to_features(unhappy_directory_path + current_file))
                 
                 
                    
                
                
                 
            
                # train single "unhappy" example
                _, logits = sess.run([optimizer, Logits], feed_dict={x: accoustic_features,
                                                          y: Y})
            
                 
                print(logits)    
        
                continue
            else:
                continue
         # Save model weights to disk
     #   save_path = saver.save(sess, model_path)
     #   print("Model saved in file: %s" % save_path)
        
        
         
    
   

  

16_39_55174.wav
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   

[[  5.89007325e-03  -4.51603672e-04   8.36827443e-04  -8.18120781e-04
   -4.62424941e-04   6.34662225e-04   3.06090733e-05   8.27987024e-05
   -4.93376865e-04   1.50149863e-04   1.33520996e-04   6.22282038e-04
   -8.72931909e-04   7.20102689e-05  -2.74922844e-04  -4.81487688e-04
    2.37831977e-04  -1.48109306e-04  -1.03279104e-04   8.15694162e-04
   -7.59306247e-04  -3.18527513e-04   1.98822439e-04   4.36959119e-04
   -2.28931473e-04   1.07912917e-03   4.75550769e-05   3.17464670e-04
   -2.98148312e-04  -5.22931456e-04  -1.14331767e-03   1.07298349e-03
   -5.38982044e-04   8.29697994e-04  -3.61111364e-04   4.43061697e-04
   -3.96317737e-05   4.60071955e-04  -8.69833399e-04  -3.46965680e-05
    2.96479673e-04   1.09108805e-03  -5.38502762e-04   9.82508878e-04
    4.49491548e-04  -7.94943946e-04  -6.31994859e-04   3.05554538e-04
   -4.20016702e-04  -2.78919731e-04   1.70582091e-04  -3.50189395e-04
    3.08338931e-05   3.81131045e-04  -1.08797403e-05   7.85556505e-04
    1.88255071e-04  

[[  1.18131945e-02  -9.07111738e-04   1.68630341e-03  -1.63763412e-03
   -9.27558751e-04   1.27422926e-03   6.36556288e-05   1.64268640e-04
   -9.94318747e-04   3.00734304e-04   2.69487704e-04   1.24259782e-03
   -1.75191485e-03   1.47518920e-04  -5.46906784e-04  -9.67692409e-04
    4.76538087e-04  -2.93915858e-04  -2.06121855e-04   1.63359323e-03
   -1.52745331e-03  -6.39301259e-04   3.98455013e-04   8.69340554e-04
   -4.58364462e-04   2.16513895e-03   9.47353838e-05   6.35773526e-04
   -6.00890664e-04  -1.04607537e-03  -2.29528872e-03   2.15638522e-03
   -1.08160719e-03   1.66483561e-03  -7.27121602e-04   8.87992675e-04
   -7.53666682e-05   9.22634266e-04  -1.74309430e-03  -7.45700527e-05
    5.92582044e-04   2.19082530e-03  -1.07945816e-03   1.97011232e-03
    9.01849824e-04  -1.59475009e-03  -1.26506377e-03   6.16138626e-04
   -8.39032698e-04  -5.58841624e-04   3.42742191e-04  -7.03682599e-04
    5.94797821e-05   7.66926096e-04  -2.45152805e-05   1.57845614e-03
    3.73678049e-04  

[[  1.77677833e-02  -1.36664091e-03   2.54851021e-03  -2.45839916e-03
   -1.39529328e-03   1.91862392e-03   9.90298649e-05   2.44474591e-04
   -1.50299503e-03   4.51737287e-04   4.07994114e-04   1.86078192e-03
   -2.63682241e-03   2.26531993e-04  -8.15800508e-04  -1.45856990e-03
    7.16117094e-04  -4.37454582e-04  -3.08562332e-04   2.45352788e-03
   -2.30447087e-03  -9.62337479e-04   5.98898623e-04   1.29709288e-03
   -6.88313216e-04   3.25789140e-03   1.41447192e-04   9.55018448e-04
   -9.08237824e-04  -1.56945991e-03  -3.45587474e-03   3.25009925e-03
   -1.62772951e-03   2.50542210e-03  -1.09802082e-03   1.33466872e-03
   -1.07316053e-04   1.38769450e-03  -2.61965278e-03  -1.19700831e-04
    8.88134877e-04   3.29915900e-03  -1.62275741e-03   2.96271150e-03
    1.35690521e-03  -2.39932328e-03  -1.89912808e-03   9.31797898e-04
   -1.25699735e-03  -8.39690445e-04   5.16398926e-04  -1.06054603e-03
    8.58865969e-05   1.15756143e-03  -4.09578824e-05   2.37864885e-03
    5.56068902e-04  

KeyboardInterrupt: 